In [ ]:
from wizard.calculator import MaterialCalculator
from wizard.atoms import SymbolInfo
from wizard.io import read_xyz
from ase.visualize import view
from pynep.calculate import NEP
from PIL import Image, ImageTk
from tkinter import ttk
import tkinter as tk
import shutil
import glob
import os
img_labels = []

# Initialize the main window
root = tk.Tk()
root.geometry("800x800")    # Set the window size to 800x800 pixels
root.title("GPUMD-wizard")  # Set the window title to "GPUMD-wizard"

# Create the Notebook widget
notebook = ttk.Notebook(root)
page1 = ttk.Frame(notebook)
page2 = ttk.Frame(notebook)
page3 = ttk.Frame(notebook)

# Add pages to the notebook
notebook.add(page1, text='Calculating')
notebook.add(page2, text='Generating')
notebook.add(page3, text='Searching')
notebook.pack(expand=True, fill='both')

# Common component creation functions
def create_label(parent, text, row, column, label_width=20, label_height=1, colspan=1, rowspan=1):
    """Create and grid a label in a specified location with given text."""
    label = tk.Label(parent, text=text, width=label_width, height=label_height)
    label.grid(row=row, column=column, columnspan=colspan, rowspan=rowspan)
    return label

def create_combobox(parent, values, row, column, combo_width=5, default_value=None):
    """Create and grid a combobox with specified values and default value."""
    combobox = ttk.Combobox(parent, values=values, width=combo_width)
    combobox.grid(row=row, column=column)
    if default_value:
        combobox.set(default_value)
    return combobox

def create_entry(parent, row, column, entry_width=5, default_value=""):
    """Create and grid an entry with a specified default value."""
    entry = tk.Entry(parent, width=entry_width)
    entry.grid(row=row, column=column)
    entry.insert(0, default_value)
    return entry

# Components for page 1
for i in range(6):
    page1.grid_columnconfigure(i, weight=1)

create_label(page1, "Formular:", 0, 0)
formular_combobox = create_combobox(page1, ["Ag", "Al", "Au", "Cr", "Cu", "Mg", "Mo", "Ni", "Pb", "Pd", "Pt", "Ta", "Ti", "V", "W", "Zr"], 0, 1, default_value="W")

create_label(page1, "Structure:", 1, 0)
structure_combobox = create_combobox(page1, ["bcc", "fcc", "hcp"], 1, 1, default_value="bcc")

create_label(page1, "Lattice Constant:", 2, 0)
lattice_constant_entry_a = create_entry(page1, 2, 1, default_value="3")
lattice_constant_entry_c = tk.Entry(page1)
lattice_constant_entry_c.insert(0, "5")

create_label(page1, "Properties:", 0, 3, colspan=2)
properties_combobox = create_combobox(page1, ["Lattice_Constant", "Elastic Modulus", "Eos Curve", "Phonon Dispersion", "Vacancy Formation Energy", "Vacancy Migration Energy", "Divacancies Formation Energy", "SIA Formation Energy", "Surface Formation Energy"], 0, 5, combo_width=20, default_value="Lattice_Constant")

# Dynamically update lattice constant entries
def update_lattice_constant_entries(event=None):
    """Show or hide the lattice constant entry_c based on the selected structure."""
    if structure_combobox.get() == "hcp":
        lattice_constant_entry_c.grid(row=2, column=2)
        lattice_constant_entry_c.configure(width=5)
    else:
        lattice_constant_entry_c.grid_remove()  # 移除c轴的输入框

structure_combobox.bind('<<ComboboxSelected>>', update_lattice_constant_entries)

property_functions = {
    "Lattice_Constant": MaterialCalculator.lattice_constant,
    "Elastic Modulus": MaterialCalculator.elastic_constant,
    "Eos Curve": MaterialCalculator.eos_curve,
    "Phonon Dispersion": MaterialCalculator.phonon_dispersion,
    "Vacancy Formation Energy": MaterialCalculator.formation_energy_vacancy,
    "Vacancy Migration Energy": MaterialCalculator.migration_energy_vacancy,
    "Divacancies Formation Energy": MaterialCalculator.formation_energy_divacancies,
    "SIA Formation Energy": MaterialCalculator.formation_energy_sia,
    "Surface Formation Energy": MaterialCalculator.formation_energy_surface
}

def create_image_label(page, img_path, row, column, columnspan):
    img = Image.open(img_path)
    tk_img = ImageTk.PhotoImage(img)
    img_label = tk.Label(page, image=tk_img)
    img_label.image = tk_img  
    img_label.grid(row=row, column=column, columnspan=columnspan) 
    return img_label

def read_and_display_file():
    output_text.delete(1.0, tk.END)
    if os.path.exists('MaterialProperties.out'):
        with open('MaterialProperties.out', 'r') as file:
            content = file.read()
    output_text.insert(tk.END, content)
    output_text.see(tk.END)

def load_atoms():
    if os.path.exists('MaterialProperties.xyz'):
        frames = read_xyz('MaterialProperties.xyz')
        view(frames)
        
def Caculation_Material_Properties(property):
    global img_labels

    calc = NEP('../Repository/UNEP_v1/nep.txt')
    symbol = formular_combobox.get()
    structure = structure_combobox.get()
    if structure == "hcp":
        symbol_info = SymbolInfo(symbol, structure, float(lattice_constant_entry_a.get()), float(lattice_constant_entry_c.get()))
    else:
        symbol_info = SymbolInfo(symbol, structure, float(lattice_constant_entry_a.get()))
    atoms = symbol_info.create_bulk_atoms()
    material_calculator = MaterialCalculator(atoms, calc, symbol_info)

    property_function = property_functions[property]
    property_function(material_calculator)

    if property in ["Eos Curve", "Phonon Dispersion"]:
        img_path = os.path.join(f"eos_curve_png", f"{symbol_info.formula}_eos_curve.png") if property == "Eos Curve" else f"{symbol_info.formula}_phono.png"
        img_label = create_image_label(page1, img_path, 3, 0, 6)
        img_labels.append(img_label)

    read_and_display_file()

def clear_output():
    global img_labels

    if os.path.exists('MaterialProperties.out'):
        os.remove('MaterialProperties.out')
    if os.path.exists('MaterialProperties.xyz'):
        os.remove('MaterialProperties.xyz')
    if os.path.exists('eos_curve_png'):
        shutil.rmtree('eos_curve_png')
    if os.path.exists('eos_curve_out'):
        shutil.rmtree('eos_curve_out')
    for png_file in glob.glob('*.png'):
        os.remove(png_file)
    
    output_text.delete(1.0, tk.END)
    for img_label in img_labels:
        img_label.destroy()
    img_labels = [] 

load_button = tk.Button(page1, text="Load", command=load_atoms)
load_button.grid(row=1, column=3, rowspan=2)
load_button.configure(height=2, width=5)

clear_button = tk.Button(page1, text="Clear", command=clear_output)
clear_button.grid(row=1, column=4, rowspan=2)
clear_button.configure(height=2, width=5)

calculate_button = tk.Button(page1, text="Calculate Material Properties", command=lambda: Caculation_Material_Properties(properties_combobox.get()))
calculate_button.grid(row=1, column=5, rowspan=2)
calculate_button.configure(height=2, width=20)

# Create an output box
output_text = tk.Text(root)
output_text.pack(side=tk.BOTTOM, fill=tk.BOTH)

root.mainloop()